In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-08,France,140930.0,0.0,26244.0,0.0,0.0,0.0
0,2020-05-08,China,84059.0,0.0,4662.0,0.0,0.0,0.0
0,2020-05-08,Italy,217307.0,0.0,30326.0,0.0,0.0,0.0
0,2020-05-08,Spain,222338.0,0.0,26294.0,0.0,0.0,0.0
0,2020-05-08,United States,1282648.0,0.0,78069.0,0.0,0.0,0.0
0,2020-05-08,World,3902831.0,0.0,275545.0,0.0,0.0,0.0
0,2020-05-08,United Kingdom,213241.0,0.0,31225.0,0.0,0.0,0.0
0,2020-05-08,Germany,168312.0,0.0,7402.0,0.0,0.0,0.0
0,2020-05-08,Iran,104803.0,0.0,6570.0,0.0,0.0,0.0
0,2020-05-08,Turkey,135805.0,0.0,3696.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-09,France,140930.0,138421.0,26244.0,26230.0,2509.0,14.0
0,2020-05-09,China,84059.0,83976.0,4662.0,4637.0,83.0,25.0
0,2020-05-09,Italy,217307.0,217185.0,30326.0,30201.0,122.0,125.0
0,2020-05-08,Spain,222338.0,222857.0,26294.0,26251.0,-519.0,43.0
0,2020-05-09,United States,1282648.0,1283929.0,78069.0,77180.0,-1281.0,889.0
0,2020-05-09,World,3902831.0,3898658.0,275545.0,274290.0,4173.0,1255.0
0,2020-05-09,United Kingdom,213241.0,211364.0,31225.0,31241.0,1877.0,-16.0
0,2020-05-09,Germany,168312.0,168551.0,7402.0,7369.0,-239.0,33.0
0,2020-05-09,Iran,104803.0,104691.0,6570.0,6541.0,112.0,29.0
0,2020-05-09,Turkey,135805.0,135569.0,3696.0,3689.0,236.0,7.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")